# Leçon 6 — Architectures complexes : ResNet, GAN, Inception

## Objectifs
- Comprendre ResNet (skip), GAN (jeu minmax), Inception (branches parallèles)
- Savoir quand utiliser chaque famille


In [ ]:
# --- Bloc résiduel (ResNet) pédagogique ---
import torch
import torch.nn as nn

class BasicBlock(nn.Module):
    def __init__(self, in_ch, out_ch, stride=1):
        super().__init__()
        self.conv1 = nn.Conv2d(in_ch, out_ch, 3, stride=stride, padding=1, bias=False)
        self.bn1   = nn.BatchNorm2d(out_ch)
        self.relu  = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_ch, out_ch, 3, stride=1, padding=1, bias=False)
        self.bn2   = nn.BatchNorm2d(out_ch)
        self.downsample = None
        if stride != 1 or in_ch != out_ch:
            self.downsample = nn.Sequential(
                nn.Conv2d(in_ch, out_ch, 1, stride=stride, bias=False),
                nn.BatchNorm2d(out_ch)
            )

    def forward(self, x):
        identity = x
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        if self.downsample is not None:
            identity = self.downsample(x)
        out += identity
        out = self.relu(out)
        return out

x = torch.randn(2, 64, 56, 56)
block = BasicBlock(64, 64)
y = block(x)
print(y.shape)


In [ ]:
# --- GAN jouet (MNIST-like, une itération pédagogique) ---
import torch
import torch.nn as nn
import torch.optim as optim

latent_dim = 64
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(latent_dim, 256), nn.ReLU(True),
            nn.Linear(256, 512), nn.ReLU(True),
            nn.Linear(512, 28*28), nn.Tanh()
        )
    def forward(self, z):
        return self.net(z).view(-1,1,28,28)

class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Flatten(),
            nn.Linear(28*28, 512), nn.LeakyReLU(0.2, True),
            nn.Linear(512, 256), nn.LeakyReLU(0.2, True),
            nn.Linear(256, 1), nn.Sigmoid()
        )
    def forward(self, x):
        return self.net(x)

G, D = Generator(), Discriminator()
opt_G = optim.Adam(G.parameters(), lr=2e-4, betas=(0.5,0.999))
opt_D = optim.Adam(D.parameters(), lr=2e-4, betas=(0.5,0.999))
bce = nn.BCELoss()
real_data = torch.randn(64,1,28,28).clamp(-1,1)
batch_size = real_data.size(0)
opt_D.zero_grad()
z = torch.randn(batch_size, latent_dim)
fake = G(z).detach()
pred_real = D(real_data)
pred_fake = D(fake)
loss_D = bce(pred_real, torch.ones_like(pred_real)) + bce(pred_fake, torch.zeros_like(pred_fake))
loss_D.backward(); opt_D.step()
opt_G.zero_grad()
z = torch.randn(batch_size, latent_dim)
fake = G(z)
pred = D(fake)
loss_G = bce(pred, torch.ones_like(pred))
loss_G.backward(); opt_G.step()
print(f'loss_D={loss_D.item():.3f} | loss_G={loss_G.item():.3f}')


In [ ]:
# --- Inception block (Keras) simplifié ---
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

def inception_block(x, filters_1x1=32, filters_3x3=32, filters_5x5=16, filters_pool=16):
    b1 = layers.Conv2D(filters_1x1, 1, activation='relu', padding='same')(x)
    b2 = layers.Conv2D(filters_3x3, 3, activation='relu', padding='same')(x)
    b3 = layers.Conv2D(filters_5x5, 5, activation='relu', padding='same')(x)
    b4 = layers.MaxPooling2D(pool_size=3, strides=1, padding='same')(x)
    b4 = layers.Conv2D(filters_pool, 1, activation='relu', padding='same')(b4)
    return layers.Concatenate()([b1,b2,b3,b4])

inp = keras.Input(shape=(64,64,3))
x = inception_block(inp)
x = layers.GlobalAveragePooling2D()(x)
out = layers.Dense(10, activation='softmax')(x)
model = keras.Model(inp, out)
model.summary()
